In [ ]:
import json
from pathlib import Path

import geopandas
import numpy
import matplotlib.pyplot as plt
import pandas
from scipy.integrate import trapezoid

In [ ]:
with open("../config.json", "r") as fh:
    config = json.load(fh)
base_path = Path(config["base_path"])
base_path

In [ ]:
with open("../scratch/polder_site_relations.json", "r") as fh:
    polder_site_relations = json.load(fh)

In [ ]:
import itertools
polder_site_link = []
for rel in polder_site_relations:
    for polder, site in itertools.product(rel['polder_set'], rel['sites_protecting']):
        polder_site_link.append({'site': site, 'polder': polder})
polder_site_link = pandas.DataFrame(polder_site_link)[['polder','site']].sort_values(by=['polder', 'site']).drop_duplicates()
polder_site_link.to_csv("../scratch/polder_site_relations.csv", index=False)

In [ ]:
polder_rp_damages = pandas.read_csv("../scratch/polder_rp_damages.csv")
polder_rp_damages.head(1)

In [ ]:
def calculate_ead(df):
    df[1] = 0
    df[float("inf")] = df[100]
    rp_cols = sorted(list(df.columns), key=lambda col: 1 / col)
    rps = numpy.array([float(col) for col in rp_cols])
    probabilities = 1 / rps
    rp_damages = df[rp_cols]
    return trapezoid(rp_damages, x=probabilities, axis=1)

In [ ]:
pivoted = polder_rp_damages.pivot(
    index=["polder_ID", "hazard", "sector", "epoch", "rcp"],
    columns="rp",
    values="damage_euro",
).replace(float("NaN"), 0)
pivoted["ead_euro"] = calculate_ead(pivoted)
polder_ead_by_sector = pivoted[["ead_euro"]].reset_index()

In [ ]:
polder_ead_by_sector.to_csv(
    "../scratch/polder_ead_by_sector.csv", index=False, float_format="%.2f"
)

In [ ]:
polder_ead_current_by_sector = polder_ead_by_sector.query('(rcp == "baseline") & (ead_euro > 0)').drop(columns=["epoch","rcp"])
polder_ead_current_by_sector.to_csv(
    "../scratch/polder_ead_current_by_sector.csv", index=False, float_format="%.2f"
)

In [ ]:
all_polders = polder_ead_current_by_sector.drop(columns=["hazard"]).groupby(["polder_ID", "sector"]).max().reset_index().drop(columns=["polder_ID"]).groupby("sector").sum()
all_polders.columns = ["EAD (2022 Euro)"]
all_polders.index = all_polders.reset_index().sector.str.title().str.replace("_", " ")
all_polders.index.name = ""
fig = all_polders.plot(kind="bar",logy=True, ylabel="EAD (2022 Euro)", legend=False).get_figure()


In [ ]:
fig.savefig("../scratch/polder_all_ead.png", bbox_inches="tight")

In [ ]:
polder_ead = (
    polder_ead_by_sector.drop(columns="sector")
    .groupby(["polder_ID", "hazard", "epoch", "rcp"])
    .sum()
)

In [ ]:
polder_ead.to_csv("../scratch/polder_ead.csv", float_format="%.2f")

In [ ]:
polder_max_current_benefit = (
    polder_ead.reset_index()
    .query("rcp == 'baseline'")
    .drop(columns=["epoch", "rcp", "hazard"])
    .groupby("polder_ID")
    .max()
)

In [ ]:
polder_max_current_benefit.to_csv(
    "../scratch/polder_ead_current_max.csv", float_format="%.2f"
)

In [ ]:
fig = polder_max_current_benefit.sort_values(by="polder_ID").plot(kind="bar", legend=False, xlabel="Polder ID", ylabel="EAD (2022 Euro)", fontsize=8).get_figure()

In [ ]:
fig.savefig("../scratch/polder_ead.png", bbox_inches="tight")

In [ ]:
polder_max_current_benefit.sum() / 1e6

In [ ]:
site_benefit = []
site_individual_benefit = []
for p in polder_site_relations:
    polder_ids = p["polder_set"]
    site_ids = p["sites_protecting"]
    max_benefit = polder_max_current_benefit.loc[polder_ids].ead_euro.sum()
    site_benefit.append(
        {
            "sites_protecting": site_ids,
            "polders_protected": polder_ids,
            "combined_protected_polder_exposure__ead_euro": max_benefit,
        }
    )
    for site in site_ids:
        site_individual_benefit.append(
            {
                "site_protecting": site,
                "polders_protected": polder_ids,
                "combined_protected_polder_exposure__ead_euro": max_benefit,
            }
        )
site_benefit = pandas.DataFrame(site_benefit).sort_values(
    by="combined_protected_polder_exposure__ead_euro", ascending=False
)
site_individual_benefit = pandas.DataFrame(site_individual_benefit).sort_values(
    by="combined_protected_polder_exposure__ead_euro", ascending=False
)

In [ ]:
site_benefit.to_csv("../scratch/site_benefit.csv", index=False, float_format="%.2f")
site_individual_benefit.to_csv(
    "../scratch/site_benefit_individual.csv", index=False, float_format="%.2f"
)

In [ ]:
sites_with_polders = geopandas.read_file(
    "../scratch/potential_sites_with_polders.gpkg", engine="pyogrio"
)

In [ ]:
sites_with_polder_benefits = sites_with_polders.merge(
    site_individual_benefit,
    how="left",
    left_on="potential_site_id",
    right_on="site_protecting",
    validate="one_to_one",
).drop(columns="site_protecting")

In [ ]:
sites_with_polder_benefits.to_file(
    "../scratch/potential_sites_with_polder_exposure.gpkg", engine="pyogrio"
)

## Copy results from scratch directory

In [ ]:
import shutil
from glob import glob

results_path = base_path / "impact" / "mangrove_protection_potential"
results_path.mkdir(exist_ok=True, parents=True)

In [ ]:
shutil.copy("../scratch/potential_sites_with_polder_exposure.gpkg", results_path)
for fname in glob("../scratch/polder_*"):
    shutil.copy(fname, results_path)

In [ ]:
glob(str(results_path / "*"))